In [2]:
from selenium import webdriver
import time
from time import sleep
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import requests
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
headers = {
    'authority': 'scrapeme.live',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

In [155]:
driver=webdriver.Chrome()
driver.get('https://www.zomato.com/bangalore')
# time.sleep(2)  # Allow 2 seconds for the web page to (open depends on you)
scroll_pause_time = 3  # You can set your own pause time. dont slow too slow that might not able to load more data
screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the user's webpage
i = 1

#Code for scrolling mechanism
while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
    print('currently scrolling screen - ',i)
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break
    
# creating soup
soup = BeautifulSoup(driver.page_source, "html.parser")
divs = soup.findAll('div', class_='jumbo-tracker')

# create all the list here according to data needs
urls = []
rest_name = []
ratings = []
price = []
cuisine = []
area=[]

for parent in divs:  
    #  name of the restaurant is stored in the h4 tags and luckily it was unique in main class
    name_tag = parent.find("h4")
    # appending the name to rest_name list
    rest_name.append(name_tag.text)

    #  links of the restaurants are in a tags hence we are using it find and then getting href where links are stored
    link_tag = parent.find("a")

    base = "https://www.zomato.com"  # since we don not get whole link https attached we need to join the link with base
    try:
        if 'href' in link_tag.attrs:
            link = link_tag.get('href')
    except:
        pass
    url = urljoin(base, link)
    urls.append(url)
    
for div in divs:
    rating_tag = div.div.a.next_sibling.div.div.div.div.div.div.div.text
    price_tag = div.div.a.next_sibling.p.next_sibling.text
    cuisine_tag = div.div.a.next_sibling.p.text
    area_tag=div.div.a.next_sibling.div.next_sibling.next_sibling.p.text
    ratings.append(rating_tag)
    price.append(price_tag)
    cuisine.append(cuisine_tag)
    area.append(area_tag)
    
area_new=[area.replace(', Bangalore','').strip() for area in area]
area_new
    
out_df = pd.DataFrame({'links': urls, 'names': rest_name, 'ratings': ratings, 'price for two': price, 'cuisine': cuisine, 'Location':area_new})
# we need to create a data frame to neatly view the data in csv format, just add the lists below

# noinspection PyTypeChecker
out_df.to_csv("Bangalore_restaurants.csv")
#  creating csv for information
driver.close()
    

currently scrolling screen -  1
currently scrolling screen -  2
currently scrolling screen -  3
currently scrolling screen -  4
currently scrolling screen -  5
currently scrolling screen -  6
currently scrolling screen -  7
currently scrolling screen -  8
currently scrolling screen -  9
currently scrolling screen -  10
currently scrolling screen -  11
currently scrolling screen -  12
currently scrolling screen -  13
currently scrolling screen -  14
currently scrolling screen -  15
currently scrolling screen -  16
currently scrolling screen -  17
currently scrolling screen -  18
currently scrolling screen -  19
currently scrolling screen -  20
currently scrolling screen -  21
currently scrolling screen -  22
currently scrolling screen -  23
currently scrolling screen -  24
currently scrolling screen -  25
currently scrolling screen -  26
currently scrolling screen -  27
currently scrolling screen -  28
currently scrolling screen -  29
currently scrolling screen -  30
currently scrolling

Cleaning data

In [126]:
urls=final_df['links'].to_list()
urls

['https://www.zomato.com/bangalore/jamming-goat-3-0-indiranagar-bangalore/info',
 'https://www.zomato.com/bangalore/hydra-club-kitchen-koramangala-5th-block-bangalore/info',
 'https://www.zomato.com/bangalore/scottish-pub-st-marks-road/info',
 'https://www.zomato.com/bangalore/raahi-st-marks-road-bangalore/info',
 'https://www.zomato.com/bangalore/oney-brewhouse-cunningham-road-bangalore/info',
 'https://www.zomato.com/bangalore/bombay-borough-richmond-road-bangalore/info',
 'https://www.zomato.com/bangalore/over-coffee-cafe-wine-bar-lavelle-road-bangalore/info',
 'https://www.zomato.com/bangalore/the-bier-library-koramangala-6th-block-bangalore/info',
 'https://www.zomato.com/bangalore/tandoor-a-heritage-indian-restaurant-bar-mg-road-bangalore/info',
 'https://www.zomato.com/bangalore/shakesbierre-1-brigade-road-bangalore/info',
 'https://www.zomato.com/bangalore/no-limmits-lounge-and-club-brigade-road-bangalore/info',
 'https://www.zomato.com/bangalore/long-boat-brewing-co-marathahal

In [4]:
final_df=pd.read_csv('Bangalore_restaurants.csv')
final_df
# final_df=out_df.copy()

,Unnamed: 0,links,names,ratings,price for two,cuisine,Location
0,0,https://www.zomato.com/bangalore/jamming-goat-...,Jamming Goat 3.0,4.4,"₹1,500 for two","South Indian, Asian, Thai",Indiranagar
1,1,https://www.zomato.com/bangalore/hydra-club-ki...,Hydra Club & Kitchen,New,"₹1,200 for two",Continental,Koramangala 5th Block
2,2,https://www.zomato.com/bangalore/scottish-pub-...,Scottish Pub,4.3,₹700 for two,Finger Food,St. Marks Road
3,3,https://www.zomato.com/bangalore/raahi-st-mark...,Raahi,4.2,"₹2,000 for two","North Indian, Pasta, Pizza, Desserts, Beverages",St. Marks Road
4,4,https://www.zomato.com/bangalore/oney-brewhous...,Oney Brewhouse,New,"₹1,500 for two","American, Continental, Italian, South Indian, ...",Cunningham Road
...,...,...,...,...,...,...,...
496,496,https://www.zomato.com/bangalore/the-coffee-br...,The Coffee Brewery,3.9,₹800 for two,"Cafe, Coffee, Sandwich, Fast Food, Desserts, T...",Koramangala 4th Block
497,497,https://www.zomato.com/bangalore/gopizza-1-whi...,GOPIZZA,4.0,₹500 for two,"Pizza, Fast Food","Phoenix Marketcity Bangalore, Whitefield"
498,498,https://www.zomato.com/bangalore/spice-it-ibis...,Spice It - Ibis Hotel,3.7,"₹1,800 for two","North Indian, Chinese, Italian, Continental, P...",Hebbal
499,499,https://www.zomato.com/bangalore/cafe-capricci...,Cafe Capriccio,4.1,₹800 for two,"Italian, Pasta, Pizza, Salad","RMZ Infinity, Old Madras Road"


converting price column from string to integer

In [5]:
final_df['price for two'] = final_df['price for two'].str.replace(' for two', '')
final_df['price for two'] = final_df['price for two'].str.replace('₹', '')
final_df['price for two'] = final_df['price for two'].str.replace(',', '')
final_df['price for two']=final_df['price for two'].astype(int)

Remvoing duplicates

In [6]:
duplicate = final_df[final_df.duplicated(['names', 'cuisine'])]
index_names = duplicate.index
print(index_names)

Index([34, 36, 38, 40, 42, 60, 62, 64], dtype='int64')


In [7]:
final_df.drop(index_names, inplace = True)

In [8]:
duplicates = final_df[final_df.duplicated(['names', 'cuisine'])] 
print(duplicates)

Empty DataFrame
Columns: [Unnamed: 0, links, names, ratings, price for two, cuisine, Location]
Index: []


Cleaning the location column to keep only broad location choices

In [9]:
final_df['area']=''
final_df.loc[final_df['Location'].str.contains('Koramangala', case=False),'area']='Koramangala'
final_df.loc[final_df['Location'].str.contains('Race Course Road', case=False),'area']='Race Course Road'
final_df.loc[final_df['Location'].str.contains('Goldfinch', case=False),'area']='Race Course Road'
final_df.loc[final_df['Location'].str.contains('Sankey Road', case=False),'area']='Race Course Road'
final_df.loc[final_df['Location'].str.contains('Old Madras Road', case=False),'area']='Old Madras Road'
final_df.loc[final_df['Location'].str.contains('Whitefield', case=False),'area']='Whitefield'
final_df.loc[final_df['Location'].str.contains('Craftworks', case=False),'area']='Whitefield'
final_df.loc[final_df['Location'].str.contains('Indiranagar', case=False),'area']='Indiranagar'
final_df.loc[final_df['Location'].str.contains('Brookefield', case=False),'area']='Brookefield'
final_df.loc[final_df['Location'].str.contains('Kanakapura Road', case=False),'area']='Kanakapura Road'
final_df.loc[final_df['Location'].str.contains('JP Nagar', case=False),'area']='JP Nagar'
final_df.loc[final_df['Location'].str.contains('Jayanagar', case=False),'area']='Jayanagar'
final_df.loc[final_df['Location'].str.contains('HSR', case=False),'area']='HSR'
final_df.loc[final_df['Location'].str.contains('Sarjapur', case=False),'area']='Sarjapur'
final_df.loc[final_df['Location'].str.contains('MG Road', case=False),'area']='MG Road'
final_df.loc[final_df['Location'].str.contains('Church Street', case=False),'area']='Church Street'
final_df.loc[final_df['Location'].str.contains('Cunningham Road', case=False),'area']='Cunningham Road'
final_df.loc[final_df['Location'].str.contains('St. Marks Road', case=False),'area']='St. Marks Road'
final_df.loc[final_df['Location'].str.contains('Hebbal', case=False),'area']='Hebbal'
final_df.loc[final_df['Location'].str.contains('Brigade Road', case=False),'area']='Brigade Road'
final_df.loc[final_df['Location'].str.contains('Richmond Road', case=False),'area']='Richmond Road'
final_df.loc[final_df['Location'].str.contains('Lavelle Road', case=False),'area']='Lavelle Road'
final_df.loc[final_df['Location'].str.contains('Marathahalli', case=False),'area']='Marathahalli'
final_df.loc[final_df['Location'].str.contains('Ulsoor', case=False),'area']='Ulsoor'
final_df.loc[final_df['Location'].str.contains('BTM', case=False),'area']='BTM'
final_df.loc[final_df['Location'].str.contains('Kalyan Nagar', case=False),'area']='Kalyan Nagar'
final_df.loc[final_df['Location'].str.contains('Bannerghatta Road', case=False),'area']='Bannerghatta'
final_df.loc[final_df['Location'].str.contains('Vasanth Nagar', case=False),'area']='Vasanth Nagar'
final_df.loc[final_df['Location'].str.contains('New BEL Road', case=False),'area']='New BEL Road'
final_df.loc[final_df['Location'].str.contains('Garuda Mall', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('Forum Rex Walk', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('UB City', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('Shanti Nagar', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('The Chancery Pavilion', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('Barton Centre', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('The Ritz-Carlton', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('1 Sobha', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('Richmond Town', case=False),'area']='Ashok Nagar'
final_df.loc[final_df['Location'].str.contains('Old Airport Road', case=False),'area']='Old Airport Road'
final_df.loc[final_df['Location'].str.contains('Basavanagudi', case=False),'area']='Basavanagudi'
final_df.loc[final_df['Location'].str.contains('Electronic City', case=False),'area']='Electronic City'
final_df.loc[final_df['Location'].str.contains('Hennur', case=False),'area']='Hennur'
final_df.loc[final_df['Location'].str.contains('Hosur Road', case=False),'area']='Hosur Road'
final_df.loc[final_df['Location'].str.contains('Bommanahalli', case=False),'area']='Hosur Road'
final_df.loc[final_df['Location'].str.contains('Bellandur', case=False),'area']='Bellandur'
final_df.loc[final_df['Location'].str.contains('Yelahanka', case=False),'area']='Yelahanka'
final_df.loc[final_df['Location'].str.contains('Brigade Gateway', case=False),'area']='Malleshwaram'
final_df.loc[final_df['Location'].str.contains('Malleshwaram', case=False),'area']='Malleshwaram'
final_df.loc[final_df['Location'].str.contains('Rajajinagar', case=False),'area']='Malleshwaram'
final_df.loc[final_df['Location'].str.contains('Lulu', case=False),'area']='Malleshwaram'
final_df.loc[final_df['Location'].str.contains('World Trade Center', case=False),'area']='Malleshwaram'
final_df.loc[final_df['Location'].str.contains('Domlur', case=False),'area']='Domlur'
final_df.loc[final_df['Location'].str.contains('Kadubeesanahalli', case=False),'area']='Marathahalli'
final_df.loc[final_df['Location'].str.contains('Hebbal', case=False),'area']='Hebbal'
final_df.loc[final_df['Location'].str.contains('RMZ Azure', case=False),'area']='Hebbal'
final_df.loc[final_df['Location'].str.contains('Hotel Country Inn', case=False),'area']='Hebbal'
final_df.loc[final_df['Location'].str.contains('Livin Hub Hotel', case=False),'area']='Marathahalli'
final_df.loc[final_df['Location'].str.contains('Sahakara Nagar', case=False),'area']='Sahakara Nagar'
final_df.loc[final_df['Location'].str.contains('Rajarajeshwari Nagar', case=False),'area']='Rajarajeshwari Nagar'
final_df.loc[final_df['Location'].str.contains('Nagawara', case=False),'area']='Nagawara'
final_df.loc[final_df['Location'].str.contains('Manyata', case=False),'area']='Nagawara'
final_df.loc[final_df['Location'].str.contains('Bhartiya', case=False),'area']='Bhartiya City'
final_df.loc[final_df['Location'].str.contains('Nagarbhavi', case=False),'area']='Nagarbhavi'	
final_df.loc[final_df['Location'].str.contains('Kanakapura', case=False),'area']='Kanakapura'


final_df[final_df['Location'].str.contains('airport', case=False)]
# final_df

,Unnamed: 0,links,names,ratings,price for two,cuisine,Location,area
20,20,https://www.zomato.com/bangalore/koko-airport-...,KOKO,4.4,3500,"Asian, Sushi, Desserts, Chinese, Beverages",Old Airport Road,Old Airport Road
97,97,https://www.zomato.com/bangalore/gawky-goose-a...,Gawky Goose,4.4,2200,"Italian, Biryani, North Indian, Fast Food, Sea...",Old Airport Road,Old Airport Road
213,213,https://www.zomato.com/bangalore/jamavar-the-l...,Jamavar - The Leela Palace,4.3,5000,"North Indian, South Indian","The Leela Palace, Old Airport Road",Old Airport Road


In [10]:
final_df=final_df[final_df['area']!='']
final_df

,Unnamed: 0,links,names,ratings,price for two,cuisine,Location,area
0,0,https://www.zomato.com/bangalore/jamming-goat-...,Jamming Goat 3.0,4.4,1500,"South Indian, Asian, Thai",Indiranagar,Indiranagar
1,1,https://www.zomato.com/bangalore/hydra-club-ki...,Hydra Club & Kitchen,New,1200,Continental,Koramangala 5th Block,Koramangala
2,2,https://www.zomato.com/bangalore/scottish-pub-...,Scottish Pub,4.3,700,Finger Food,St. Marks Road,St. Marks Road
3,3,https://www.zomato.com/bangalore/raahi-st-mark...,Raahi,4.2,2000,"North Indian, Pasta, Pizza, Desserts, Beverages",St. Marks Road,St. Marks Road
4,4,https://www.zomato.com/bangalore/oney-brewhous...,Oney Brewhouse,New,1500,"American, Continental, Italian, South Indian, ...",Cunningham Road,Cunningham Road
...,...,...,...,...,...,...,...,...
496,496,https://www.zomato.com/bangalore/the-coffee-br...,The Coffee Brewery,3.9,800,"Cafe, Coffee, Sandwich, Fast Food, Desserts, T...",Koramangala 4th Block,Koramangala
497,497,https://www.zomato.com/bangalore/gopizza-1-whi...,GOPIZZA,4.0,500,"Pizza, Fast Food","Phoenix Marketcity Bangalore, Whitefield",Whitefield
498,498,https://www.zomato.com/bangalore/spice-it-ibis...,Spice It - Ibis Hotel,3.7,1800,"North Indian, Chinese, Italian, Continental, P...",Hebbal,Hebbal
499,499,https://www.zomato.com/bangalore/cafe-capricci...,Cafe Capriccio,4.1,800,"Italian, Pasta, Pizza, Salad","RMZ Infinity, Old Madras Road",Old Madras Road


Converting cuisine into a list

In [133]:
final_df[final_df['names']=='Bologna']

,Unnamed: 0,links,names,ratings,price for two,cuisine,Location,area
122,122,https://www.zomato.com/bangalore/bologna-indir...,Bologna,4.8,"₹1,500 for two","Italian, Pasta, Pizza, Desserts",Indiranagar,Indiranagar


In [11]:
# Create a new column containing lists of cuisine values
# final_df['cuisine_list'] = final_df['cuisine'].str.lower().apply(lambda x: x.split(', '))
final_df['area_list']=final_df['area']
final_df['cuisine_list']=final_df['cuisine']
final_df['area']=final_df['area'].str.lower()
final_df['cuisine']=final_df['cuisine'].str.lower()
final_df['cuisine']=final_df['cuisine'].str.replace(',','')
final_df['area']=final_df['area'].str.replace(' ','')
final_df

C:\Users\sunil\AppData\Local\Temp\ipykernel_34632\547025145.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['area_list']=final_df['area']
C:\Users\sunil\AppData\Local\Temp\ipykernel_34632\547025145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['cuisine_list']=final_df['cuisine']
C:\Users\sunil\AppData\Local\Temp\ipykernel_34632\547025145.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

,Unnamed: 0,links,names,ratings,price for two,cuisine,Location,area,area_list,cuisine_list
0,0,https://www.zomato.com/bangalore/jamming-goat-...,Jamming Goat 3.0,4.4,1500,south indian asian thai,Indiranagar,indiranagar,Indiranagar,"South Indian, Asian, Thai"
1,1,https://www.zomato.com/bangalore/hydra-club-ki...,Hydra Club & Kitchen,New,1200,continental,Koramangala 5th Block,koramangala,Koramangala,Continental
2,2,https://www.zomato.com/bangalore/scottish-pub-...,Scottish Pub,4.3,700,finger food,St. Marks Road,st.marksroad,St. Marks Road,Finger Food
3,3,https://www.zomato.com/bangalore/raahi-st-mark...,Raahi,4.2,2000,north indian pasta pizza desserts beverages,St. Marks Road,st.marksroad,St. Marks Road,"North Indian, Pasta, Pizza, Desserts, Beverages"
4,4,https://www.zomato.com/bangalore/oney-brewhous...,Oney Brewhouse,New,1500,american continental italian south indian nort...,Cunningham Road,cunninghamroad,Cunningham Road,"American, Continental, Italian, South Indian, ..."
...,...,...,...,...,...,...,...,...,...,...
496,496,https://www.zomato.com/bangalore/the-coffee-br...,The Coffee Brewery,3.9,800,cafe coffee sandwich fast food desserts tea be...,Koramangala 4th Block,koramangala,Koramangala,"Cafe, Coffee, Sandwich, Fast Food, Desserts, T..."
497,497,https://www.zomato.com/bangalore/gopizza-1-whi...,GOPIZZA,4.0,500,pizza fast food,"Phoenix Marketcity Bangalore, Whitefield",whitefield,Whitefield,"Pizza, Fast Food"
498,498,https://www.zomato.com/bangalore/spice-it-ibis...,Spice It - Ibis Hotel,3.7,1800,north indian chinese italian continental pasta...,Hebbal,hebbal,Hebbal,"North Indian, Chinese, Italian, Continental, P..."
499,499,https://www.zomato.com/bangalore/cafe-capricci...,Cafe Capriccio,4.1,800,italian pasta pizza salad,"RMZ Infinity, Old Madras Road",oldmadrasroad,Old Madras Road,"Italian, Pasta, Pizza, Salad"


In [12]:
final_df.reset_index(drop=True, inplace=True)
final_df

,Unnamed: 0,links,names,ratings,price for two,cuisine,Location,area,area_list,cuisine_list
0,0,https://www.zomato.com/bangalore/jamming-goat-...,Jamming Goat 3.0,4.4,1500,south indian asian thai,Indiranagar,indiranagar,Indiranagar,"South Indian, Asian, Thai"
1,1,https://www.zomato.com/bangalore/hydra-club-ki...,Hydra Club & Kitchen,New,1200,continental,Koramangala 5th Block,koramangala,Koramangala,Continental
2,2,https://www.zomato.com/bangalore/scottish-pub-...,Scottish Pub,4.3,700,finger food,St. Marks Road,st.marksroad,St. Marks Road,Finger Food
3,3,https://www.zomato.com/bangalore/raahi-st-mark...,Raahi,4.2,2000,north indian pasta pizza desserts beverages,St. Marks Road,st.marksroad,St. Marks Road,"North Indian, Pasta, Pizza, Desserts, Beverages"
4,4,https://www.zomato.com/bangalore/oney-brewhous...,Oney Brewhouse,New,1500,american continental italian south indian nort...,Cunningham Road,cunninghamroad,Cunningham Road,"American, Continental, Italian, South Indian, ..."
...,...,...,...,...,...,...,...,...,...,...
469,496,https://www.zomato.com/bangalore/the-coffee-br...,The Coffee Brewery,3.9,800,cafe coffee sandwich fast food desserts tea be...,Koramangala 4th Block,koramangala,Koramangala,"Cafe, Coffee, Sandwich, Fast Food, Desserts, T..."
470,497,https://www.zomato.com/bangalore/gopizza-1-whi...,GOPIZZA,4.0,500,pizza fast food,"Phoenix Marketcity Bangalore, Whitefield",whitefield,Whitefield,"Pizza, Fast Food"
471,498,https://www.zomato.com/bangalore/spice-it-ibis...,Spice It - Ibis Hotel,3.7,1800,north indian chinese italian continental pasta...,Hebbal,hebbal,Hebbal,"North Indian, Chinese, Italian, Continental, P..."
472,499,https://www.zomato.com/bangalore/cafe-capricci...,Cafe Capriccio,4.1,800,italian pasta pizza salad,"RMZ Infinity, Old Madras Road",oldmadrasroad,Old Madras Road,"Italian, Pasta, Pizza, Salad"


In [108]:
final_df.to_csv('zomato_restaurant_final.csv')

In [46]:
final_df['cuisine'].str.split().explode().unique()

array(['south', 'indian', 'asian', 'thai', 'continental', 'finger',
       'food', 'north', 'pasta', 'pizza', 'desserts', 'beverages',
       'american', 'italian', 'chinese', 'biryani', 'tea', 'european',
       'cafe', 'coffee', 'bar', 'oriental', 'bbq', 'mughlai', 'kebab',
       'kashmiri', 'seafood', 'grilled', 'chicken', 'fast', 'salad',
       'street', 'modern', 'sushi', 'lebanese', 'greek', 'moroccan',
       'indonesian', 'andhra', 'mediterranean', 'japanese', 'burger',
       'sandwich', 'mangalorean', 'cantonese', 'korean', 'mexican',
       'sichuan', 'paan', 'bakery', 'burmese', 'bubble', 'ice', 'cream',
       'tex-mex', 'steak', 'shake', 'healthy', 'panini', 'rolls',
       'malaysian', 'hyderabadi', 'kerala', 'vietnamese', 'pancake',
       'goan', 'lucknowi', 'parsi', 'turkish', 'middle', 'eastern',
       'gujarati', 'rajasthani', 'brazilian', 'momos', 'wraps', 'french',
       'juices', 'awadhi', 'mithai', 'portuguese', 'arabian', 'peruvian',
       'waffle', 'shawa

Using a Word2Vec model

In [227]:
import scipy.linalg

In [13]:
import scipy

print("SciPy version:", scipy.__version__)

SciPy version: 1.12.0


In [55]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.preprocessing import normalize

# Concatenate 'area' and 'cuisine' columns to create a corpus of text data
corpus = final_df['area'] + ' ' + final_df['cuisine']

# Tokenize the corpus into lists of words
tokenized_corpus = [doc.split() for doc in corpus]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Get embedding vectors for words
word_vectors = word2vec_model.wv

# Convert each document into a vector representation by averaging word vectors
document_vectors = []
for doc in tokenized_corpus:
    doc_vector = [word_vectors[word] for word in doc if word in word_vectors]
    if doc_vector:
        document_vector = np.mean(doc_vector, axis=0)
        document_vectors.append(document_vector)
    else:
        document_vectors.append([0] * 100)  # If document is empty, use zero vector

# Normalize document vectors
X_normalized = normalize(document_vectors, norm='l2', axis=1)

# Convert normalized vectors to DataFrame for display
word_embedding_df = pd.DataFrame(X_normalized, columns=[f'feature_{i+1}' for i in range(100)])

display(word_embedding_df)


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100
0,-0.011403,0.070492,0.041050,-0.002747,0.035474,-0.127951,0.118308,0.228196,-0.163086,-0.155453,...,0.176090,0.044808,0.036721,-0.016403,0.148100,0.061765,0.054137,-0.070156,-0.004282,-0.037986
1,-0.028450,0.075265,0.028488,0.000887,0.032357,-0.125113,0.106220,0.231588,-0.172880,-0.135543,...,0.171291,0.044689,0.032990,-0.025441,0.139078,0.054952,0.067189,-0.055072,0.001334,-0.028052
2,-0.024230,0.067464,0.037744,-0.020716,0.018673,-0.135578,0.107910,0.223628,-0.170987,-0.154102,...,0.187235,0.049372,0.018075,-0.022661,0.150265,0.060731,0.067688,-0.076747,-0.007186,-0.038697
3,-0.017420,0.073830,0.032715,-0.002454,0.036845,-0.132647,0.113562,0.232820,-0.166666,-0.155659,...,0.171482,0.049010,0.037066,-0.012316,0.150947,0.055060,0.056102,-0.070855,-0.007321,-0.036559
4,-0.018214,0.070100,0.035604,0.001169,0.034090,-0.126063,0.115498,0.229598,-0.169469,-0.152867,...,0.171793,0.042541,0.036189,-0.014372,0.145145,0.056584,0.056895,-0.069879,-0.003430,-0.033960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,-0.016556,0.070781,0.032867,-0.008436,0.031037,-0.131804,0.111237,0.233089,-0.165688,-0.154488,...,0.170311,0.048079,0.030853,-0.014505,0.147169,0.058338,0.058075,-0.072436,-0.005797,-0.040741
470,-0.016170,0.067295,0.032340,-0.004255,0.025975,-0.131256,0.109272,0.232993,-0.167890,-0.159951,...,0.172471,0.056003,0.031086,-0.003620,0.151773,0.054697,0.055194,-0.074017,-0.003689,-0.044099
471,-0.018016,0.075132,0.032184,-0.001590,0.038579,-0.126002,0.114747,0.230766,-0.167600,-0.151543,...,0.171561,0.046096,0.036220,-0.012640,0.147884,0.058372,0.058007,-0.069117,-0.006303,-0.037333
472,-0.016782,0.073275,0.030515,-0.001318,0.038768,-0.124817,0.123284,0.230256,-0.168663,-0.155254,...,0.173225,0.054180,0.037342,0.000882,0.151756,0.051844,0.050857,-0.061200,-0.001275,-0.031242


In [63]:
import pickle

# Save Word2Vec model to a pickle file
with open('word_vectors.pkl', 'wb') as f:
    pickle.dump(word_vectors, f)

In [115]:
import pickle

# Save Word2Vec model to a pickle file
with open('document_vectors.pkl', 'wb') as f:
    pickle.dump(document_vectors, f)

In [64]:
# Load the object from the pickle file
with open('word_vectors.pkl', 'rb') as f:
    word_vectors = pickle.load(f)

In [116]:
# Load the object from the pickle file
with open('document_vectors.pkl', 'rb') as f:
    document_vectors = pickle.load(f)

In [110]:
from sklearn.metrics.pairwise import cosine_similarity

# Tokenize the query into a list of words
query = "koramangala continental"
query_tokens = query.split()

# Check if each word in the query exists in the vocabulary of the Word2Vec model
valid_tokens = [word for word in query_tokens if word in word_vectors]

if valid_tokens:
    # Calculate the average vector representation of the query
    query_vector = np.mean([word_vectors[word] for word in valid_tokens], axis=0).reshape(1, -1)

    # Compute cosine similarity between the query vector and document vectors
    similarities = cosine_similarity(query_vector, document_vectors)

    # Get indices of documents or items sorted by similarity scores (descending order)
    similar_indices = np.argsort(similarities[0])[::-1]

    # Retrieve top-N most similar matches
    top_n = 8
    similar_matches = final_df.iloc[similar_indices[:top_n]]

    print("Top {} similar matches to '{}'".format(top_n, query))
    display(similar_matches)
else:
    print("None of the words in the query are present in the Word2Vec vocabulary.")


Top 8 similar matches to 'koramangala continental'


,Unnamed: 0,links,names,ratings,price for two,cuisine,Location,area,area_list,cuisine_list
1,1,https://www.zomato.com/bangalore/hydra-club-ki...,Hydra Club & Kitchen,New,1200,continental,Koramangala 5th Block,koramangala,Koramangala,Continental
449,474,https://www.zomato.com/bangalore/ganbeii-koram...,Ganbeii,New,1000,continental asian,Koramangala 5th Block,koramangala,Koramangala,"Continental, Asian"
272,284,https://www.zomato.com/bangalore/kobe-sizzlers...,Kobe Sizzlers,3.9,1600,pasta burger sandwich american continental,Koramangala 5th Block,koramangala,Koramangala,"Pasta, Burger, Sandwich, American, Continental"
160,171,https://www.zomato.com/bangalore/fennys-lounge...,Fenny's Lounge And Kitchen,4.5,1900,mediterranean continental italian desserts bev...,Koramangala 7th Block,koramangala,Koramangala,"Mediterranean, Continental, Italian, Desserts,..."
161,172,https://www.zomato.com/bangalore/the-hole-in-t...,The Hole In The Wall Cafe,4.4,700,cafe continental burger sandwich shake beverag...,Koramangala 4th Block,koramangala,Koramangala,"Cafe, Continental, Burger, Sandwich, Shake, Be..."
377,397,https://www.zomato.com/bangalore/blurred-koram...,Blurred,4.1,1000,north indian south indian continental chinese,Koramangala 5th Block,koramangala,Koramangala,"North Indian, South Indian, Continental, Chinese"
230,241,https://www.zomato.com/bangalore/sugar-factory...,Sugar Factory Reloaded,4.0,1800,north indian continental italian chinese asian...,Koramangala 5th Block,koramangala,Koramangala,"North Indian, Continental, Italian, Chinese, A..."
200,211,https://www.zomato.com/bangalore/rooftop-at-ra...,Rooftop at Raw,4.0,1200,continental italian mexican pizza pasta beverages,Koramangala 5th Block,koramangala,Koramangala,"Continental, Italian, Mexican, Pizza, Pasta, B..."


In [113]:
similar_indices = np.argsort(similarities[0])[::-1]
similar_indices=np.argsort(similarities[0])[::-1][:3]
similar_indices

array([  1, 449, 272], dtype=int64)

In [72]:
dummy=pd.read_csv('zomato_restaurant_final.csv')
dummy1=dummy[dummy['area'].str.contains('jpnagar')]
dummy1[dummy1['cuisine'].str.contains('asian')]

,Unnamed: 0.1,Unnamed: 0,links,names,ratings,price for two,cuisine,Location,area,area_list,cuisine_list
22,22,23,https://www.zomato.com/bangalore/uru-brewpark-...,Uru Brewpark,4.3,"₹2,200 for two",continental north indian italian chinese asian...,JP Nagar,jpnagar,JP Nagar,"Continental, North Indian, Italian, Chinese, A..."
23,23,24,https://www.zomato.com/bangalore/ambedo-kitche...,Ambedo Kitchen|Taproom,4.5,"₹1,500 for two",continental asian north indian italian pizza s...,JP Nagar,jpnagar,JP Nagar,"Continental, Asian, North Indian, Italian, Piz..."
41,41,47,https://www.zomato.com/bangalore/butterfly-bre...,Butterfly Brew Bite & Fly,4.2,"₹1,500 for two",north indian continental pizza italian asian d...,JP Nagar,jpnagar,JP Nagar,"North Indian, Continental, Pizza, Italian, Asi..."
59,59,68,https://www.zomato.com/bangalore/terrace-by-ho...,Terrace By House Of Commons,4.6,"₹2,000 for two",modern indian continental asian chinese north ...,JP Nagar,jpnagar,JP Nagar,"Modern Indian, Continental, Asian, Chinese, No..."
69,69,79,https://www.zomato.com/bangalore/indian-biere-...,Indian Biere House,4.4,"₹2,500 for two",continental mediterranean north indian asian b...,JP Nagar,jpnagar,JP Nagar,"Continental, Mediterranean, North Indian, Asia..."
73,73,83,https://www.zomato.com/bangalore/the-scene-cra...,The Scene - Craft Kitchen & Bar,4.2,"₹1,500 for two",north indian continental asian desserts beverages,JP Nagar,jpnagar,JP Nagar,"North Indian, Continental, Asian, Desserts, Be..."
86,86,96,https://www.zomato.com/bangalore/brahma-brews-...,Brahma Brews,4.5,"₹2,500 for two",asian north indian south indian,JP Nagar,jpnagar,JP Nagar,"Asian, North Indian, South Indian"
105,105,115,https://www.zomato.com/bangalore/the-pump-hous...,The Pump House,4.4,"₹2,500 for two",continental chinese north indian italian asian...,JP Nagar,jpnagar,JP Nagar,"Continental, Chinese, North Indian, Italian, A..."
108,108,118,https://www.zomato.com/bangalore/levels-pub-ki...,Levels Pub & Kitchen,4.4,"₹1,700 for two",asian south indian north indian,JP Nagar,jpnagar,JP Nagar,"Asian, South Indian, North Indian"
172,172,183,https://www.zomato.com/bangalore/khmer-kitchen...,Khmer Kitchen,4.6,"₹2,500 for two",chinese asian thai japanese,JP Nagar,jpnagar,JP Nagar,"Chinese, Asian, Thai, Japanese"


In [155]:
from sklearn.preprocessing import normalize

Trying TFIDF

In [78]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(final_df['area'] + ' ' + final_df['cuisine'])

# Convert to DataFrame for display
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Normalize TF-IDF matrix
X_normalized = normalize(X, norm='l2', axis=1)

display(tfidf_df)

,afghan,american,andhra,arabian,ashoknagar,asian,awadhi,bakery,bannerghatta,bar,...,tex,thai,turkish,ulsoor,vasanthnagar,vietnamese,waffle,whitefield,wraps,yelahanka
0,0.0,0.0000,0.0,0.0,0.0,0.333679,0.0,0.0,0.0,0.0,...,0.0,0.587325,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
2,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
3,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
4,0.0,0.4181,0.0,0.0,0.0,0.239530,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
470,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.64951,0.0,0.0
471,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
472,0.0,0.0000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [208]:
tfidf_df_input = pd.DataFrame(user_input_vector.toarray(), columns=vectorizer.get_feature_names_out())

In [212]:
print(tfidf_df.loc[313]['sushi'])
print(tfidf_df.loc[453]['sushi'])
print(tfidf_df_input['sushi'])


0.0
0.36124120168978446
0    0.704211
Name: sushi, dtype: float64


In [149]:
tfidf_df.loc[20]['jayanagar']

0.8422874172145479

In [150]:
tfidf_df.loc[62]['jayanagar']

0.43278666095878693

In [102]:
location = 'koramangala'
cuisine='continental'

#Pre-processing input
location=location.replace(' ','')
user_input = location + ' ' + cuisine
final_output=('').join(user_input.lower().split('  '))


# TF-IDF Vectorization of user input
user_input_vector = vectorizer.transform([final_output])

# Calculate cosine similarity between user input and recipes
similarity_scores = cosine_similarity(user_input_vector, X_normalized)

# Get indices of top recommended recipes (top 3 in this example)
top_indices = similarity_scores.argsort()[0][-4:][::-1]
top_indices

array([  1, 449, 446, 373], dtype=int64)

In [105]:
final_df.loc[446]

Unnamed: 0                                                     471
links            https://www.zomato.com/bangalore/mooch-marod-k...
names                                                  Mooch Marod
ratings                                                        4.3
price for two                                                  600
cuisine                                     north indian beverages
Location                                     Koramangala 6th Block
area                                                   koramangala
area_list                                              Koramangala
cuisine_list                               North Indian, Beverages
Name: 446, dtype: object

In [181]:
dummy = pd.DataFrame(user_input_vector.toarray(), columns=vectorizer.get_feature_names_out())
dummy['sushi']

0    0.704211
Name: sushi, dtype: float64

In [189]:
final_df.loc[313]

Unnamed: 0                                                     329
links            https://www.zomato.com/bangalore/sattvam-jp-na...
names                                                      Sattvam
ratings                                                        4.2
price for two                                       ₹1,960 for two
cuisine                                        chinese continental
Location                                                  JP Nagar
area                                                       jpnagar
area_list                                                 JP Nagar
cuisine_list                                  Chinese, Continental
Name: 313, dtype: object

Saving the vector in pickle files 

In [756]:
import pickle

# File path to save X
x_file_path = "concatenated_features.pkl"

# Save X as a pickle file
with open(x_file_path, 'wb') as file:
    pickle.dump(X, file)

# File path to save X
vector_file_path = "vectorizer.pkl"

# Save X as a pickle file
pickle.dump(vectorizer, open(vector_file_path, "wb"))

Recommendation function

In [30]:
import pickle

In [32]:
def recommend_sys(location , cuisine):
    
    # Calling TFIDF pickle files
    vectorizer = pickle.load(open("vectorizer.pkl", 'rb'))

    # Load the transformed X vector from the pickle file
    with open("concatenated_features.pkl", "rb") as file:
        X = pickle.load(file)
    
    # Convert to DataFrame for display
    tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
    
    #Pre-processing input
    location=location.replace(' ','')
    user_input = location + ' ' + cuisine
    final_output=('').join(user_input.lower().split('  '))
    

    # TF-IDF Vectorization of user input
    user_input_vector = vectorizer.transform([final_output])

    # Calculate cosine similarity between user input and recipes
    similarity_scores = cosine_similarity(user_input_vector, X)

    # Get indices of top recommended recipes (top 3 in this example)
    top_indices = similarity_scores.argsort()[0][-3:][::-1]
    
    recommendations = "\nTop Recommended Restaurants:\n"
    for idx in top_indices:
        recommendations += f"Restaurant Name: {final_df.iloc[idx]['names']}\n"
        recommendations += f"Location: {final_df.iloc[idx]['area']}\n"
        recommendations += f"Cuisine: {final_df.iloc[idx]['cuisine']}\n"
        recommendations += f"Rating: {final_df.iloc[idx]['ratings']}\n"
        recommendations += f"Cost for two: {final_df.iloc[idx]['price for two']}\n"
        recommendations += f"Similarity Score: {round(similarity_scores[0][idx],2)}\n\n"
    
    return recommendations  

In [115]:
output=recommend_sys('JP Nagar', 'mughlai')
print(output)


Top Recommended Restaurants:
Restaurant Name: 1522 - The Pub
Location: jpnagar
Cuisine: chinese north indian mughlai finger food seafood
Rating: 4.4
Cost for two: ₹1,800 for two
Similarity Score: 0.64

Restaurant Name: Sattvam
Location: jpnagar
Cuisine: chinese continental
Rating: 4.2
Cost for two: ₹1,960 for two
Similarity Score: 0.61

Restaurant Name: Three Dots & A Dash
Location: jpnagar
Cuisine: north indian continental italian
Rating: 4.3
Cost for two: ₹1,800 for two
Similarity Score: 0.54




In [752]:
# location = 'indiranagar'
# cuisine = 'pizza'
# user_input = location + ' ' + cuisine
# preprocessed_input = input_preprocessor(user_input)

# # TF-IDF Vectorization of user input
# user_input_vector = vectorizer.transform([preprocessed_input])

# # Calculate cosine similarity between user input and recipes
# similarity_scores = cosine_similarity(user_input_vector, X)

# # Get indices of top recommended recipes (top 3 in this example)
# top_indices = similarity_scores.argsort()[0][-3:][::-1]

# # Display top recommended recipes
# print("\nTop Recommended Recipes:")
# for idx in top_indices:
#     print("Restaurant:", final_df.loc[idx, 'names'])
#     print("Cuisine:", final_df.loc[idx, 'cuisine'])
#     print("Location:", final_df.loc[idx, 'area'])
#     print("Similarity Score:", round(similarity_scores[0][idx],2))
#     print()

In [726]:
# #pre-process inputs from user
# def input_preprocessor(input):
#     final_output=('').join(input.lower().split('  '))
#     return final_output

Trying TF-IDF vectorizer

In [698]:
# def recommend(description):
#     description = description.lower()

#     data = final_df.copy()
    
#     # Extract cities
#     area_input = []
#     for area in area_list:
#         if area in description:
#             area_input.append(area)
#             description = description.replace(area, "")

#     if area_input:
#         data = data[data['area'].isin(area_input)]
        
#     description=input_preprocessor(description)
#     print('Processed user feedback:', description)
    
#     vectorizer = TfidfVectorizer()
#     X = vectorizer.fit_transform(data['cuisine'])
    
#     # Transform user input data based on fitted model
#     description_vector =  vectorizer.transform([description])
    
#     # Calculate cosine similarity between user input and recipes
#     similarity_score = cosine_similarity(description_vector, X)
    
#     top_indices = similarity_score.argsort()[0][-3:][::-1]
    
#     # Sort top_indices based on ratings
#     top_indices_sorted_by_ratings = sorted(top_indices, key=lambda idx: data.iloc[idx]['ratings'], reverse=True)
    
    
#     recommendations = "\nTop Recommended Restaurants:\n"
#     for idx in top_indices_sorted_by_ratings:
#         recommendations += f"Restaurant Name: {data.iloc[idx]['names']}\n"
#         recommendations += f"Location: {data.iloc[idx]['area']}\n"
#         recommendations += f"Cuisine: {data.iloc[idx]['cuisine']}\n"
#         recommendations += f"Rating: {data.iloc[idx]['ratings']}\n"
#         recommendations += f"Cost for two: {data.iloc[idx]['price for two']}\n"
#         recommendations += f"Similarity Score: {round(similarity_score[0][idx],2)}\n\n"
    
#     return recommendations
    

In [123]:
# cuisine = 'asian'
# location='vasanthnagar'
# user_input=location.replace(' ','')+ ' ' + cuisine

# recommendations=recommend(user_input)
# print(recommendations)

In [141]:
# zomato = pd.read_csv('Bangalore_restaurants.csv')
# must_try_final = []
# links = []
# for i in range(len(urls)):
#     print(zomato['links'][i])
#     # try:
#     link = zomato['links'][i]
#     links.append(link)
#     linkpage = requests.get(link, headers=headers)
#     # print(i,".",link)
#     soup = BeautifulSoup(linkpage.text, 'html.parser')
#     sideCol = soup.find('p', class_=['sc-1hez2tp-0', 'sc-dRFBHB', 'gjUzYI'])
#     maps_direction=soup.find_all('href', class_='sc-kxYOAa jtaDZv')
#     for ele in sideCol:
#         print(ele.text)
#         # print(sideCol)
#     # dc = {}
#     # for i in sideCol:
#     #     val = ""
#     #     if '(' in i.text:
#     #         s1 = len(i.text)
#     #         s = i.text
#     #         for x in range(s1 - 2, 0, -1):
#     #             if (s[x] == '('):
#     #                 break
#     #             val = s[x] + val
#     #         dc[((s.split('('))[0]).rstrip()] = int(val)
#     # resmenu.append(dc)
